In [ ]:
import pandas as pd 
import numpy as np
import asyncio
from requests_html import AsyncHTMLSession
import json
import os

In [ ]:
asession = AsyncHTMLSession()
materials = []
df = pd.read_csv('Zara_product_list.csv')
task = df['productId']
task

In [ ]:

async def link(val, counter = [0]):
    counter[0] += 1
    m = counter[0]
    if m % 250 == 0 and m != 0:
        print(m)
        print(val)
        await asyncio.sleep(1)
    l = await asession.get(f"https://www.zara.com/in/en/product/{val}/extra-detail?ajax=true")
    return l
    
def detail(val):
        value = val.get('text', {}).get('value')
        return value 
    
for ids in task:        
    r = await link(ids)
    if(r.status_code == 200):            
        json_string = r.text
        if json_string.strip():  # Check if the content is not empty
            try:
                data = json.loads(json_string)
                result_dict = {}
                for packet in data: 
                    item = packet.get('components', [])
                    check = packet.get("sectionType")
                    if(check == 'materials'):                    
                        scale = len(item)
                        first = []
                        second = []
                        for j in range(4,scale):
                            text = item[j].get('datatype')
                            if(text == 'subtitle'):                        
                                h = detail(item[j])
                                first.append(h)                            
                            elif(text == 'paragraph'):
                                b = detail(item[j])
                                second.append(b)
                        result_dict = dict(zip(first, second))
                    elif(check == 'origin'):
                        scale = len(item) - 1
                        origin = detail(item[scale])
                        result_dict.update({"origins" : origin, "productId" : ids})                             
                        materials.append(result_dict)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                print(len(materials))
        else:
            print("Empty response or invalid JSON content")
            print(len(materials))
    else:
        print(f"Request failed with status code: {r.status_code}")
        print(len(materials))           

In [ ]:
len(materials)

In [ ]:
df = pd.DataFrame(materials)
df

In [ ]:
df.to_csv('ZaraMat.csv')

In [ ]:

# Check for columns with a single unique value
single_value_cols = []
for col in df.columns:
    if df[col].nunique() == 1:
        single_value_cols.append(col)

if single_value_cols:
    print("Columns with a single value:")
    print(single_value_cols)
else:
    print("No columns with a single value found.")


In [ ]:
dd.tail(30)